In [411]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('netflix.sqlite')

In [412]:
c = conn.cursor()

In [ ]:
def dict_factory(cursor, row):
    # обертка для преобразования
    # полученной строки. (взята из документации)
    d = {}
    for idx, col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d
conn.row_factory = dict_factory
c = conn.cursor()  # Исправлено на большую "C"
c.execute("SELECT * FROM netflix_titles limit 3")
result = c.fetchall()  # Теперь будут словари
print(result)

In [414]:
con = sqlite3.connect('task1.sqlite')
curs = con.cursor()

In [415]:
curs.execute('''
CREATE TABLE IF NOT EXISTS actors (
    show_id INTEGER,
    actor_name TEXT
)
''')

In [416]:
query = 'INSERT INTO actors (show_id, actor_name) VALUES (?, ?)'
actors = []
types = c.execute('''SELECT show_id, "cast" FROM netflix_titles''')
for actor in types:
    actor_values = actor.values()
    actors_names_str = list(actor_values)[1]
    actors_show_id = list(actor_values)[0]
    actors_names_list = actors_names_str.split(', ')
    for actor_name in actors_names_list:
        actors.append((actors_show_id, actor_name,))
with con:
    curs.executemany(query, actors)



In [ ]:
pd.read_sql("SELECT * FROM actors", con)

In [ ]:
curs.execute('''
CREATE TABLE IF NOT EXISTS films (
    show_id INTEGER,
    film TEXT
)
''')

In [419]:
query = 'INSERT INTO films (show_id, film) VALUES (?, ?)'
flms = []
types = c.execute('''SELECT show_id, type, title FROM netflix_titles WHERE type = 'Movie' ''')
for movie in types:
    movie_values = movie.values()
    movie_id = list(movie_values)[0]
    movie_title = list(movie_values)[2]
    flms.append((movie_id, movie_title,))  # Wrap the title in a tuple

with con:
    curs.executemany(query, flms)



In [ ]:
pd.read_sql("SELECT * FROM films", con)

In [ ]:
curs.execute('''
CREATE TABLE IF NOT EXISTS actor_film (
    show_id INTEGER,
    actor_name TEXT,
    film TEXT
)
''')

In [424]:
query = "INSERT INTO actor_film (show_id, actor_name, film) VALUES (?,?,?)"
act_flm = []
types_films = curs.execute("""
SELECT actors.show_id, actors.actor_name, films.film
 FROM actors
JOIN films ON actors.show_id = films.show_id
""").fetchall()
for film_id, actor, film in types_films:
    act_flm.append((film_id, actor, film))
with con:
    curs.executemany(query, act_flm)


In [ ]:
pd.read_sql("SELECT * FROM actor_film", con)

In [426]:
con2 = sqlite3.connect('task1.sqlite')
c2 = con2.cursor()

sql_query = '''
SELECT actor1.actor_name AS actor1, actor2.actor_name AS actor2, COUNT(*) AS count
FROM actors AS actor1
JOIN actors AS actor2 ON actor1.show_id = actor2.show_id AND actor1.actor_name < actor2.actor_name
GROUP BY actor1.actor_name, actor2.actor_name
ORDER BY count DESC
LIMIT 1;
'''

# Выполнение запроса
c2.execute(sql_query)

# Получение результатов
result = c2.fetchone()

# Вывод результатов
if result:
    actor1, actor2, count = result
    print(f"Наиболее часто работающая пара актеров: {actor1} и {actor2}, количество фильмов: {count}")

Наиболее часто работающая пара актеров: John Paul Tremblay и Robb Wells, количество фильмов: 15
